Compresive Strength Concrete Problem

Import Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tp
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import models
from keras import layers
from tensorflow.keras import optimizers
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
print (tf.__version__)

2.4.1


# Load Data

In [2]:
data=pd.read_csv("/content/compresive_strength_concrete.csv")

# Data Analysis

In [3]:
data.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
data.shape

(1030, 9)

There are 9 Columns (need to be renamed). 1030 Rows of data



In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1030 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1030 non-null   float64
 7   Age (day)                                              1030 non-null   int64  
 8   Concrete compressive strength(MPa, megapascals)  

In [6]:
data.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


Data Cleaning

In [7]:
data.columns = ['Cement ','Blast_Furnace','Fly_Ash','Water','Superplasticizer','Coarse_Aggregate','Fine_Aggregate','Age','CC_Strength']

Null Values if Any

In [8]:
data.isnull().sum()

Cement              0
Blast_Furnace       0
Fly_Ash             0
Water               0
Superplasticizer    0
Coarse_Aggregate    0
Fine_Aggregate      0
Age                 0
CC_Strength         0
dtype: int64

In [9]:
np.where(data.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [10]:
x = data.drop('CC_Strength', axis=1)
y = data['CC_Strength']

# Normalizing Data with Mean-Std

In [11]:
x_train, x_test, y_train, y_test =train_test_split(x,y, test_size=0.3, random_state=42)

# assign features and target columns

In [12]:
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std
x_test -= mean
x_test /= std

In [13]:
mean_label = y_train.mean(axis=0)
y_train -= mean_label 
std_label = y_train.std(axis=0)
y_train /= std_label
y_test -= mean_label
y_test /= std_label

In [14]:
x_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Cement,721.0,-7.830075e-17,1.0,-1.683389,-0.852826,-0.068405,0.721554,2.358687
Blast_Furnace,721.0,-1.684583e-16,1.0,-0.854703,-0.854703,-0.603898,0.798331,3.242540
Fly_Ash,721.0,4.194518e-16,1.0,-0.816346,-0.816346,-0.816346,1.028017,2.244512
Water,721.0,2.044289e-15,1.0,-2.836060,-0.765956,0.184023,0.481777,3.081222
Superplasticizer,721.0,4.559460e-16,1.0,-1.013292,-1.013292,-0.020966,0.657123,4.312190
Coarse_Aggregate,721.0,-2.876879e-15,1.0,-2.200993,-0.532401,-0.073857,0.843231,2.180652
Fine_Aggregate,721.0,9.605469e-15,1.0,-2.216980,-0.590791,0.085644,0.616058,2.757686
Age,721.0,2.432944e-17,1.0,-0.730728,-0.519863,-0.292777,0.161394,5.173499


In [15]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy().astype('float32')
x_test = x_test.to_numpy()
y_test = y_test.to_numpy().astype('float32')

# Model Function (Book)

In [16]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(10, activation= 'relu',input_shape=(x_train.shape[1],)))
    model.add(layers.Dense(8, activation= 'relu'))
    model.add(layers.Dense(6, activation= 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

K-fold Validation

In [17]:
k = 4
num_val_samples = len(x_train) // k 
num_epochs = 100
all_scores_relu = []

for i in range(k):
  print('processing fold #', i)
  val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate([x_train[:i * num_val_samples],x_train[(i + 1) * num_val_samples:]],  axis=0)
  partial_train_targets = np.concatenate([y_train[:i * num_val_samples],y_train[(i + 1) * num_val_samples:]],axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores_relu.append(val_mae)


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


# Training Model

In [18]:
model = build_model()
model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=0)

# Evaluation / Prediction

In [19]:
test_mse_score, test_mae_score = model.evaluate(x_test, y_test)
print("mae score ",test_mae_score)
print("mse score ",test_mse_score)

10/10 [==============================] - 0s 2ms/step - loss: 0.1759 - mae: 0.3160
mae score  0.3160485625267029
mse score  0.175927996635437
